# Thumos14 exploration

In [25]:
from pathlib import Path
import pickle
import av
import pandas as pd

In [18]:
BASE_PATH = Path("/mnt/archive/common/datasets/thumos14")

In [19]:
ds_path = BASE_PATH / "splits/val.pickle"
assert ds_path.exists()

with open(ds_path, 'rb') as f:
    ds = pickle.load(f)

In [39]:
ds[vid_id]['anno'][0].s

False

In [31]:
# BaseballPitch: video_validation_0000683
# VoleyballSpiking: video_validation_0000987
vid_id = 'video_validation_0000283'
ds[vid_id]['anno'].shape

(308, 22)

In [10]:
df = pd.read_csv(BASE_PATH / "classes.txt" , sep=" ", header=None)
df

,0,1
0,7,BaseballPitch
1,9,BasketballDunk
2,12,Billiards
3,21,CleanAndJerk
4,22,CliffDiving
5,23,CricketBowling
6,24,CricketShot
7,26,Diving
8,31,FrisbeeCatch
9,33,GolfSwing


### Do labels match in sequence?
Col 0 indicates no action.

`[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]`


Col 1 is BaseballPitch (see e.g. video_validation_0000683)

`[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]`


Col 10 is GolfSwing (see e.g. video_test_0000847)

`[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]`


Col 20 is VoleyballSpiking (see e.g. video_validation_0000987)

`[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]`


Col 21 is Ambiguous (see e.g video_validation_0000204)

`[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]`

### What is the FPS of the annotation?

In [91]:
# First, let's check the video info
vid_path = BASE_PATH / "data" / "val" / f"{vid_id}.mp4"
assert vid_path.exists()

container = av.open(str(vid_path))
video_stream = container.streams.video[0]

{
    "size": f"{container.size} bits", 
    "duration": f"{container.duration / 1e6} seconds",
    "frames": f"{video_stream.frames} frames",
    "frame_rate": f"{float(video_stream.base_rate)} fps",
}

{'size': '28491265 bits',
 'duration': '68.543 seconds',
 'frames': '2053 frames',
 'frame_rate': '30.0 fps'}

In [93]:
# Annotation frame rate, computed two ways
num_annot_frames = len(ds[vid_id]['anno'])
(
    num_annot_frames / (container.duration / 1e6),
    float(video_stream.base_rate) * num_annot_frames / video_stream.frames
)

(4.989568591978758, 4.997564539698003)

In [94]:
# Reduction factor:
num_annot_frames / video_stream.frames

0.16658548465660009

### Precomputed features from OadTR

In [5]:
feat_path = BASE_PATH / "thumos_all_feature_val_tsn_v2.pickle"
assert feat_path.exists()

with open(feat_path, 'rb') as f:
    feat = pickle.load(f)

In [15]:
feat['video_validation_0000690']['rgb'].shape

(1401, 2048)

In [8]:
feat['video_validation_0000690']['flow'].shape

(1401, 1024)

In [11]:
feat['video_validation_0000288']['rgb'].shape

(559, 2048)

In [12]:
feat['video_validation_0000288']['flow'].shape

(559, 1024)

It looks like the precomputed features allow different feature lengths

# Dataloader

In [1]:
from datasets.thumos14 import Thumos14
from pathlib import Path

/home/lh/.local/lib/python3.8/site-packages/torchvision/transforms/_functional_video.py:5: UserWarning: The _functional_video module is deprecated. Please use the functional module instead.
  warnings.warn(
/home/lh/.local/lib/python3.8/site-packages/torchvision/transforms/_transforms_video.py:25: UserWarning: The _transforms_video module is deprecated. Please use the transforms module instead.
  warnings.warn(


In [2]:
ds = Thumos14(
        Path("/mnt/archive/common/datasets/thumos14/"),
        "/mnt/archive/common/datasets/thumos14/splits/",
        10,
    )

datasets: Using validation set for training as is customary for THUMOS14.


In [3]:
data = ds[0]

ValueError: read of closed file